## Code build and train ANN

In [ ]:
include("./code/general/utils_general.jl")

include("./code/ann/build_train.jl")
include("./code/ann/utils_ann.jl")

using Pkg; Pkg.add("DelimitedFiles");    
using DelimitedFiles

using Pkg; Pkg.add("Plots")
using Plots; Pkg.add("PyPlot")



#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, validationIndexes, testIndexes) = holdOut(datasetLength, 0.2, 0.2)
(trainDataset, validationDataset, testDataset) = (dataset[trainIndexes, :], dataset[validationIndexes, :], dataset[testIndexes, :])


(trainInput, validationInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, validationDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))


classes = unique(trainDataset[:,5])
(trainOutput, validationOutput, testOutput) = (oneHotEncoding(trainDataset[:,5], classes), oneHotEncoding(validationDataset[:,5], classes), oneHotEncoding(testDataset[:,5], classes))


# Normalization 
normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, validationInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(validationInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))


#Define the parameters
topology = [4, 3]; 
learningRate = 0.01;
numMaxEpochs = 1000; 
topologies_to_try = [[5], [10], [4, 3], [8, 4]]
    
for topo in topologies_to_try
    println("Training with topology: ", topo)
    (ann, trainingLosses, validationLosses, testLosses) = trainClassANN(topo, (trainInputNormalized, trainOutput); validationDataset=(validationInputNormalized, validationOutput), testDataset=(testInputNormalized, testOutput), maxEpochs=numMaxEpochs, learningRate=learningRate);

  g =  plot(1:length(trainingLosses), 
    trainingLosses, 
    label="Train losses", 
    xlabel="Epoch", 
    ylabel="Loss value", 
    title="Loss evolution")

    if !isempty(validationLosses)
        plot!(1:length(validationLosses), validationLosses, label="Validation losses")
    end

    if !isempty(testLosses)
        plot!(1:length(testLosses), testLosses, label="Test losses")
    end

    display(g)
end;  


## Code metrics

In [ ]:
include("./code/general/train_metrics.jl")

#Test boolean and boolean function
outputs=[0,0,1,1,0,1,1,0,0,0,1,1]
targets=[0,1,1,0,1,1,1,0,1,1,0,0]
printConfusionMatrix(convert(Array{Bool,1}, outputs), convert(Array{Bool,1}, targets));
println("\n\n")

#Test real and boolean
outputs=[0.4; 0.6; 1; 0.4;  0.55]
targets=[0;   1;   1; 1; 0]
printConfusionMatrix(convert(Array{Real,1}, outputs), convert(Array{Bool,1}, targets));


#Test array bool
testOutputs=Bool[0 0 1;1 0 0;0 1 0; 0 0 1; 0 0 1]
testTargets=Bool[0 0 1;0 1 0;0 1 0; 1 0 0; 1 0 0]
printConfusionMatrix(testOutputs, testTargets, weighted=false)
println("\n\n")

#Test array real and target bool
testRealOutputs=Real[0 0 0.7;0 0 0.6;0 1 0; 0 0 1; 0 0 1]
printConfusionMatrix(testRealOutputs, testTargets, weighted=false)
println("\n\n")

#Test array any 
result = confusionMatrix(testOutputs, testTargets)

testClasses=["dog", "cat", 3, :green, "dog", 3, "dog"]
testClassesTargets=["dog", "cat", 3, :green, "dog", 3, "dog"]
uniqueClasss=unique(testClasses)
printConfusionMatrix(testClasses, testClassesTargets, uniqueClasss)
println("\n\n")

#Test two classes 
result = confusionMatrix(testOutputs, testTargets)

testClasses=["dog"; "cat"]
testClassesTargets=["dog"; "cat"]
uniqueClasss=unique(testClasses)
printConfusionMatrix(testClasses, testClassesTargets, uniqueClasss)
println("\n\n")
println("hola")

## BUILD AND TRAIN CROSS VALIDATION ANN

In [ ]:
using Pkg; Pkg.add("DelimitedFiles");    
using DelimitedFiles

include("./code/general/utils_general.jl")
include("./code/ann/build_train.jl")
include("./code/ann/utils_ann.jl")


a=[false,true, false, true, true, false]


result = crossvalidation(5, 3)

println("Result", result)

b = crossvalidation(a, 3)
println("Cross validation AbstractArray{Bool, 1}", b)


a = Bool[0 1;1 0;1 0; 0 1;0 1]

bool2_indices = crossvalidation(a, 10)
println("Cross validation AbstractArray{Bool,2} ", bool2_indices)

unique(bool2_indices)


any_array = ['a', :b, 1, 'a']
one = oneHotEncoding(any_array)

println("Size ", size(any_array, 1), " lenght one " ,length(one), " size one ", size(one), "number columns one", size(one, 2))

any_indices = crossvalidation(any_array, 2)
println("Cross validation AbstractArray{<:Any,2} ", any_indices)


#HOW TO

b = [1 1 1 "perro"; 0 0 0 "gato"]


#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, testIndexes) = holdOut(datasetLength, 0.2)
(trainDataset, testDataset) = (dataset[trainIndexes, :], dataset[testIndexes, :])
(trainInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))



classes = unique(trainDataset[:,5])
normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))

topology = [4, 3]; 
crossValidationIndices = crossvalidation(trainDataset[:, 5], 8)


ANNCrossValidation(topology,
        (trainInputNormalized, trainDataset[:, 5]),
        crossValidationIndices)

## MLJ CROSS VALIDATION

In [6]:
import Pkg;
Pkg.add("DelimitedFiles");    
using DelimitedFiles

include("./code/general/model_factory.jl")
include("./code/ann/utils_ann.jl")
include("./code/general/utils_general.jl")



topology = [4, 3]

ann_hyperparameters = Dict(
    :modelType => :ANN,
    :topology => topology,                      
    :transferFunctions => fill(σ, length(topology)),            
    :learningRate => 0.01,                  
    :validationRatio => 0.2,                  
    :maxEpochs => 1000,          
    :maxEpochsVal => 20                         
)


svc_hyperparameters = Dict(
    :modelType => :SVC,
    :cost   => Float64(1.0),                          
    :kernel => "rbf",              
    :degree => Int32(3),                             
    :gamma  => Float64(0.1),                          
    :coef0  => Float64(0.0)                         
)

dt_hyperparameters = Dict(
    :modelType => :DecisionTreeClassifier,
    :max_depth => 5,                                     
    :rng       => Random.MersenneTwister(1)             
)

knn_hyperparameters = Dict(
    :modelType => :KNeighborsClassifier,
    :n_neighbors => 3   
)


#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, testIndexes) = holdOut(datasetLength, 0.2)
(trainDataset, testDataset) = (dataset[trainIndexes, :], dataset[testIndexes, :])
(trainInput, testInput) =  (trainDataset[:,1:4], testDataset[:,1:4])

(trainInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))

normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))

crossValidationIndices = crossvalidation(trainDataset[:, 5], 5)


result = modelCrossValidation(
    :ANN, ann_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result ANN",result)



result = modelCrossValidation(
    :SVC, svc_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result SVC",result)

result = modelCrossValidation(
    :DecisionTreeClassifier, svc_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result DecisionTreeClassifier",result)


result = modelCrossValidation(
    :KNeighborsClassifier, knn_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result KNN",result)

   Resolving package versions...


  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


   Resolving package versions...


...

0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 282: -- Training loss 0.24688928 -- Validation loss:0.33368325 -- Test loss 0.29907888
Epoch 282: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 283: -- Training loss 0.24554002 -- Validation loss:0.3326224 -- Test loss 0.2979244
Epoch 283: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 284: -- Training loss 0.24419875 -- Validation loss:0.33156696 -- Test loss 0.2967751
Epoch 284: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 285: -- Training loss 0.24286538 -- Validation loss:0.33051702 -- Test loss 0.29563105
Epoch 285: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 286: -- Training loss 0.24153997 -- Validation loss:0.32947257 -- Test loss 0.29449213
Epoch 286: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 287: -- Training los

 -- Test acc  0.92
Epoch 338: -- Training loss 0.18294808 -- Validation loss:0.28313062 -- Test loss 0.24277455
Epoch 338: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 339: -- Training loss 0.1820054 -- Validation loss:0.2823956 -- Test loss 0.24192913
Epoch 339: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 340: -- Training loss 0.18106897 -- Validation loss:0.28166628 -- Test loss 0.2410894
Epoch 340: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 341: -- Training loss 0.18013886 -- Validation loss:0.28094277 -- Test loss 0.24025537
Epoch 341: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 342: -- Training loss 0.17921495 -- Validation loss:0.28022495 -- Test loss 0.23942702
Epoch 342: -- Training acc  0.9859154929577465 -- Validation acc: 0.875 -- Test acc  0.92
Epoch 343: -- Training loss 0.17829725 -- Validation loss:0.2795128 -

1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 394: -- Training loss 0.13884257 -- Validation loss:0.2503162 -- Test loss 0.20398775
Epoch 394: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 395: -- Training loss 0.13819659 -- Validation loss:0.24987221 -- Test loss 0.20344563
Epoch 395: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 396: -- Training loss 0.13755488 -- Validation loss:0.2494328 -- Test loss 0.20290838
Epoch 396: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 397: -- Training loss 0.13691759 -- Validation loss:0.2489977 -- Test loss 0.2023759
Epoch 397: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 398: -- Training loss 0.13628444 -- Validation loss:0.24856722 -- Test loss 0.20184833
Epoch 398: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 399: -- Training loss 0.1356555 -

Excessive output truncated after 524292 bytes.

0.18661608
Epoch 432: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 433: -- Training loss 0.11656842 -- Validation loss:0.23609835 -- Test loss 0.18624245
Epoch 433: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 434: -- Training loss 0.116068654 -- Validation loss:0.23581104 -- Test loss 0.18587282
Epoch 434: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 435: -- Training loss 0.115572125 -- Validation loss:0.23552726 -- Test loss 0.18550712
Epoch 435: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 436: -- Training loss 0.11507874 -- Validation loss:0.23524696 -- Test loss 0.1851453
Epoch 436: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0.92
Epoch 437: -- Training loss 0.11458852 -- Validation loss:0.23497017 -- Test loss 0.1847874
Epoch 437: -- Training acc  1.0 -- Validation acc: 0.9166666666666666 -- Test acc  0